In [1]:
%pip install --quiet accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 3.5 MB/s eta 0:00:00


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, AutoProcessor, SeamlessM4TModel

/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


In [2]:
phi_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
phi_tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [48]:
phi_pipe = pipeline(
    "text-generation",
    model=phi_model,
    tokenizer=phi_tokenizer,
  )
generation_args = {
  "max_new_tokens": 500,
  "return_full_text": False,
  "do_sample": False
}

In [49]:
def modify_sentence(sentence):
  messages = [
    {"role" : "user", "content": f"If the sentence doesn't have any figurative or idioms, then output the sentence exactly as is. If the sentence has figurative language or idioms, output a sentence of the same meaning without any idioms or figurative language. Do not attempt to explain the meaning, just output the modified sentence. Sentence: {sentence}"},
    ]
  phi_output = phi_pipe(messages, **generation_args)
  modified_sentence = phi_output[0]['generated_text']

  print(f'Original sentence: {sentence}')
  print(f'Modified sentence: {modified_sentence}')
  return modified_sentence

In [50]:
examples = ["Let the cat out of the bag", "Piece of cake", "It's a dog-eat-dog world", "Speak of the devil", "Costs an arm and a leg", "Beating around the bush", "Caught red-handed", "Elephant in the room", "Once in a blue moon", "Let sleeping dogs lie"]

In [51]:
modified_sentences = []
for example in examples:
  modified_sentences.append(modify_sentence(example))

Original sentence: Let the cat out of the bag
Modified sentence:  Reveal the secret or confidential information.
Original sentence: Piece of cake
Modified sentence:  It is an easy task.
Original sentence: It's a dog-eat-dog world
Modified sentence:  It's a competitive world.
Original sentence: Speak of the devil
Modified sentence:  Speak now of the person you are referring to.
Original sentence: Costs an arm and a leg
Modified sentence:  The cost is very high.
Original sentence: Beating around the bush
Modified sentence:  Do not avoid the main topic directly.
Original sentence: Caught red-handed
Modified sentence:  Caught in the act of wrongdoing.
Original sentence: Elephant in the room
Modified sentence:  The issue that is obvious but not being discussed.
Original sentence: Once in a blue moon
Modified sentence:  Once it happens infrequently.
Original sentence: Let sleeping dogs lie
Modified sentence:  Do not provoke or stir up issues that are currently dormant.


In [52]:
# del all phi related vars
# del phi_model
# del phi_tokenizer
# del phi_pipe
# del generation_args
# import gc
# gc.collect()

105

In [68]:
import torch
torch.cuda.empty_cache()

In [69]:
seamless_processor = AutoProcessor.from_pretrained("facebook/hf-seamless-m4t-large", device_map='cuda')
seamless_model = SeamlessM4TModel.from_pretrained("facebook/hf-seamless-m4t-large", device_map='cuda')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [70]:
src_lang = "eng"
tgt_lang = "arb"

In [77]:
def translate_sentence(sentence, tgt_lang):
  text_inputs = seamless_processor(text=sentence, return_tensors="pt").to("cuda")
  output_tokens = seamless_model.generate(**text_inputs, tgt_lang=tgt_lang, generate_speech=False)
  translated_text = seamless_processor.decode(output_tokens[0].tolist()[0], skip_special_tokens=True)
  print(f'Translated sentence: {translated_text}')

In [81]:
for modified_sentence in modified_sentences:
  print(f'Original sentence: {examples[modified_sentences.index(modified_sentence)]}')
  print(f'Modified sentence: {modified_sentence}')
  translate_sentence(modified_sentence, tgt_lang)

Original sentence: Let the cat out of the bag
Modified sentence:  Reveal the secret or confidential information.
Translated sentence: كشف المعلومات السرية أو السرية.
Original sentence: Piece of cake
Modified sentence:  It is an easy task.
Translated sentence: إنها مهمة سهلة.
Original sentence: It's a dog-eat-dog world
Modified sentence:  It's a competitive world.
Translated sentence: إنه عالم تنافسي.
Original sentence: Speak of the devil
Modified sentence:  Speak now of the person you are referring to.
Translated sentence: تحدث الآن عن الشخص الذي تشير إليه.
Original sentence: Costs an arm and a leg
Modified sentence:  The cost is very high.
Translated sentence: التكلفة عالية جداً
Original sentence: Beating around the bush
Modified sentence:  Do not avoid the main topic directly.
Translated sentence: لا تتجنب الموضوع الرئيسي مباشرة.
Original sentence: Caught red-handed
Modified sentence:  Caught in the act of wrongdoing.
Translated sentence: تم القبض عليه في فعل خاطئ
Original sentence: 

In [82]:
tgt_lang = "urd"

In [83]:
for modified_sentence in modified_sentences:
  print(f'Original sentence: {examples[modified_sentences.index(modified_sentence)]}')
  print(f'Modified sentence: {modified_sentence}')
  translate_sentence(modified_sentence, tgt_lang)

Original sentence: Let the cat out of the bag
Modified sentence:  Reveal the secret or confidential information.
Translated sentence: خفیہ یا خفیہ معلومات کو ظاہر کریں.
Original sentence: Piece of cake
Modified sentence:  It is an easy task.
Translated sentence: یہ ایک آسان کام ہے.
Original sentence: It's a dog-eat-dog world
Modified sentence:  It's a competitive world.
Translated sentence: یہ ایک مسابقتی دنیا ہے.
Original sentence: Speak of the devil
Modified sentence:  Speak now of the person you are referring to.
Translated sentence: اب اس شخص کے بارے میں بات کریں جس کا آپ حوالہ دے رہے ہیں۔
Original sentence: Costs an arm and a leg
Modified sentence:  The cost is very high.
Translated sentence: قیمت بہت زیادہ ہے.
Original sentence: Beating around the bush
Modified sentence:  Do not avoid the main topic directly.
Translated sentence: براہ راست مرکزی موضوع سے گریز نہ کریں۔
Original sentence: Caught red-handed
Modified sentence:  Caught in the act of wrongdoing.
Translated sentence: غل